# AI-Powered Interview Assessment System

## 1. Import Libraries & Initialize Models

In [20]:
import os
import glob
import ctypes
import site
import json
import subprocess
import requests
from pathlib import Path
from tqdm import tqdm

def force_load_nvidia_libs():
    """
    Force load NVIDIA libraries (cublas & cudnn) into memory using ctypes.
    This resolves LD_LIBRARY_PATH issues in Jupyter Notebook.
    """
    try:
        site_packages = site.getsitepackages()
        libs_loaded = []
        
        for sp in site_packages:
            nvidia_dir = Path(sp) / "nvidia"
            if not nvidia_dir.exists():
                continue

            targets = [
                "cublas/lib/libcublas.so.12",
                "cudnn/lib/libcudnn_ops_infer.so.8",
                "cudnn/lib/libcudnn_cnn_infer.so.8" 
            ]
            
            for target in targets:
                lib_path = list(nvidia_dir.glob(target))
                if not lib_path:
                     lib_path = list(nvidia_dir.glob(f"*/lib/{Path(target).name}"))
                
                if lib_path:
                    try:
                        ctypes.CDLL(str(lib_path[0]))
                        libs_loaded.append(lib_path[0].name)
                    except Exception as e:
                        print(f"Warning: Failed to load {lib_path[0].name}: {e}")

        if libs_loaded:
            print(f"Successfully pre-loaded NVIDIA libraries: {', '.join(libs_loaded)}")
        else:
            print("Warning: No NVIDIA libraries found in venv. Run 'pip install nvidia-cublas-cu12'")

    except Exception as e:
        print(f"Error during library pre-load: {e}")

# Force load NVIDIA libraries before importing faster_whisper
force_load_nvidia_libs()

from faster_whisper import WhisperModel

# Setup directories
TEMP_FOLDER = "temp_interview_data"
VIDEO_FOLDER = "vid"
os.makedirs(TEMP_FOLDER, exist_ok=True)
os.makedirs(VIDEO_FOLDER, exist_ok=True)

# Load Whisper model
print("Loading Whisper model (INT8 mode for lower VRAM usage)...")
try:
    model_stt = WhisperModel("large-v3", device="cuda", compute_type="int8")
    print("Model loaded successfully (GPU/CUDA - INT8 Mode)")
except Exception as e:
    print(f"Error loading model: {e}")

Successfully pre-loaded NVIDIA libraries: libcublas.so.12
Loading Whisper model (INT8 mode for lower VRAM usage)...
Model loaded successfully (GPU/CUDA - INT8 Mode)
Model loaded successfully (GPU/CUDA - INT8 Mode)


## 2. Configure Input Data & Scoring Rubrics

In [21]:
# Input payload - interview data
input_payload = {
    "data": {
        "reviewChecklists": {
            "interviews": [
                {
                    "positionId": 1,
                    "question": "Can you share any specific challenges you faced while working on certification and how you overcame them?",
                    "recordedVideoUrl": "https://drive.google.com/uc?id=1B8vb9lYz9LCCWpVdyglbDOgg-_ncndw8" 
                },
                {
                    "positionId": 2,
                    "question": "Can you describe your experience with transfer learning in TensorFlow? How did it benefit your projects?",
                    "recordedVideoUrl": "https://drive.google.com/uc?id=1VmUhGiGpSReiQlGabzqkehfPy8cKyZzR"
                },
                {
                    "positionId": 3,
                    "question": "Describe a complex TensorFlow model you have built and the steps you took to ensure its accuracy and efficiency.",
                    "recordedVideoUrl": "https://drive.google.com/file/d/1b0R1W-FnZyziW4a4EUComRXkQhJF5gNV/view?usp=drive_link"
                },
                {
                    "positionId": 4,
                    "question": "Explain how to implement dropout in a TensorFlow model and the effect it has on training.",
                    "recordedVideoUrl": "https://drive.google.com/file/d/1iUwQxAeJpB8oB7HjyyMpfq-c_lBIfyc4/view?usp=drive_link"
                },
                {
                    "positionId": 5,
                    "question": "Describe the process of building a convolutional neural network (CNN) using TensorFlow for image classification.",
                    "recordedVideoUrl": "https://drive.google.com/file/d/1JE34UAK-NQMHyx4XPo-cmTBbZZ_tYj85/view?usp=drive_link"
                }
            ]
        },
        "certification": {
            "autoGraderProjectScore": 100
        }
    }
}

# Scoring rubrics
RUBRICS = {
    1: "Score 4 if detailed challenge & solution. Score 2 if general without detail. Score 0 if unanswered.",
    2: "Score 4 if specific transfer learning experience & benefit shown. Score 2 if general definition only.",
    3: "Score 4 if complex model architecture detailed. Score 2 if basic model mentioned.",
    4: "Score 4 if dropout implementation & effect explained clearly. Score 2 if vague.",
    5: "Score 4 if CNN building steps (layers, compile, train) detailed. Score 2 if steps missing."
}

## 3. Core Functions: Media Processing & AI Evaluation

In [22]:
def process_media(file_id):
    """
    Load video from vid folder and convert to WAV audio.
    
    Args:
        file_id: Question ID number
        
    Returns:
        tuple: (audio_path, error_message)
    """
    audio_path = os.path.join(TEMP_FOLDER, f"{file_id}.wav")
    
    # Check for video files with various formats and naming patterns
    local_video_extensions = ['.mp4', '.webm', '.mkv', '.avi', '.mov']
    video_path = None
    
    possible_names = [
        f"ans_{file_id}",
        f"interview_question_{file_id}",
        f"question_{file_id}",
        f"{file_id}"
    ]
    
    for name in possible_names:
        for ext in local_video_extensions:
            potential_path = os.path.join(VIDEO_FOLDER, f"{name}{ext}")
            if os.path.exists(potential_path):
                video_path = potential_path
                break
        if video_path:
            break
    
    if not video_path:
        supported_formats = ', '.join(local_video_extensions)
        return None, f"Video for question #{file_id} not found in '{VIDEO_FOLDER}'. Supported formats: {supported_formats}"

    # Convert video to audio (WAV 16kHz Mono)
    subprocess.run([
        "ffmpeg", "-i", video_path, 
        "-ar", "16000", "-ac", "1", 
        "-c:a", "pcm_s16le", "-y", "-vn",
        audio_path
    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    if not os.path.exists(audio_path):
        return None, f"Failed to extract audio from {video_path}"
    
    return audio_path, None


def ask_ollama(question, answer_text, rubric_text):
    """
    Send data to Ollama (Llama 3.1) for evaluation.
    
    Args:
        question: Interview question
        answer_text: Transcribed answer
        rubric_text: Scoring rubric
        
    Returns:
        dict: {"score": int, "reason": str}
    """
    system_prompt = f"""
    You are a Technical Interview Assessor. 
    Evaluate the candidate's answer based on the Rubric.
    
    Question: "{question}"
    Rubric Rule: {rubric_text}
    
    You MUST output a JSON object with strictly these fields:
    - "score": (integer 0-4)
    - "reason": (string, max 2 sentences explaining the score)
    """

    user_prompt = f"Candidate Answer: {answer_text}"

    response = requests.post('http://localhost:11434/api/chat', json={
        "model": "llama3.1",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "format": "json",
        "stream": False
    })
    
    if response.status_code == 200:
        return json.loads(response.json()['message']['content'])
    else:
        return {"score": 0, "reason": "Error calling AI"}

## 4. Process All Interview Questions

In [23]:
interview_results = []
total_interview_score = 0
interviews_data = input_payload['data']['reviewChecklists']['interviews']
transcript_details = []

print("Starting assessment process...\n")

# Process each interview question with progress bar
for item in tqdm(interviews_data, desc="Processing interviews", unit="question"):
    q_id = item['positionId']
    question = item['question']
    
    # Step 1: Load video and extract audio
    audio_file, error = process_media(q_id)
    if error:
        tqdm.write(f"[Question {q_id}] {error}")
        continue
    
    tqdm.write(f"[Question {q_id}] Video found, extracting audio...")
        
    # Step 2: Transcribe audio using Whisper
    tqdm.write(f"[Question {q_id}] Transcribing audio...")
    segments, _ = model_stt.transcribe(audio_file, beam_size=5, language=None)
    transcript_text = " ".join([s.text for s in segments])
    tqdm.write(f"[Question {q_id}] Transcript: {transcript_text[:80]}...")
    
    # Step 3: Evaluate using Ollama AI
    tqdm.write(f"[Question {q_id}] Evaluating response...")
    rubric_rule = RUBRICS.get(q_id, "General assessment")
    ai_result = ask_ollama(question, transcript_text, rubric_rule)
    
    # Step 4: Save results
    result_obj = {
        "id": q_id,
        "score": ai_result['score'],
        "reason": ai_result['reason']
    }
    interview_results.append(result_obj)
    total_interview_score += ai_result['score']
    
    # Save transcript details
    transcript_details.append({
        "video_id": q_id,
        "question": question,
        "answer": transcript_text,
        "score": ai_result['score'],
        "reason": ai_result['reason']
    })
    
    tqdm.write(f"[Question {q_id}] Score: {ai_result['score']}/4 - {ai_result['reason']}\n")

print("\nAll questions processed successfully!")

Starting assessment process...



Processing interviews:   0%|          | 0/5 [00:00<?, ?question/s]

[Question 1] Video found, extracting audio...
[Question 1] Transcribing audio...


Processing interviews:   0%|          | 0/5 [00:07<?, ?question/s]

[Question 1] Transcript:  ok, share any specific challenges you faced while working on certification and ...
[Question 1] Evaluating response...


...

[Question 4] Score: 2/4 - The candidate's answer is vague and lacks clarity in explaining the implementation of dropout in a TensorFlow model and its effect on training. They mention using dropout layers in a project, but fail to provide a clear explanation of how it works or its benefits.

[Question 5] Video found, extracting audio...
[Question 5] Transcribing audio...


Processing interviews:  80%|████████  | 4/5 [02:32<00:34, 34.18s/question]

[Question 5] Transcript:  Describe the process of building or configuring or using software image as fict...
[Question 5] Evaluating response...


Processing interviews: 100%|██████████| 5/5 [02:50<00:00, 34.10s/question]

[Question 5] Score: 2/4 - The candidate's answer is partially correct as they mention some steps in building a CNN, but it lacks detail. They outline the basic process of splitting the dataset and using data augmentation, but skip important details such as compiling the model, selecting an optimizer and loss function, and training the model.


All questions processed successfully!


## 5. Generate Final Results & Export

In [24]:
# Calculate final scores
project_score = input_payload['data']['certification']['autoGraderProjectScore']
interview_score_scaled = (total_interview_score / 20) * 100
final_total_score = (project_score + interview_score_scaled) / 2

# Determine pass/fail decision
decision = "PASSED" if final_total_score >= 75 else "Need Human Review"

# Build final JSON output
final_output = {
    "assessorProfile": {
        "id": 1,
        "name": "AI Auto-Assessor",
        "photoUrl": "https://ui-avatars.com/api/?name=AI"
    },
    "decision": decision,
    "reviewedAt": "2025-11-30 10:00:00",
    "scoresOverview": {
        "project": project_score,
        "interview": total_interview_score,
        "total": final_total_score
    },
    "reviewChecklistResult": {
        "project": [],
        "interviews": {
            "minScore": 0,
            "maxScore": 4,
            "scores": interview_results
        }
    },
    "transcriptDetails": transcript_details,
    "Overall notes": f"Assessment completed automatically via Local AI. Total Score: {final_total_score}"
}

# Save to JSON file
with open('hasil_penilaian_final.json', 'w') as f:
    json.dump(final_output, f, indent=2)

print("=" * 80)
print("ASSESSMENT RESULTS")
print("=" * 80)
print(json.dumps(final_output, indent=2))

# Display detailed transcript and scoring
print("\n" + "=" * 80)
print("DETAILED TRANSCRIPT & SCORING")
print("=" * 80 + "\n")

for detail in transcript_details:
    print(f"VIDEO #{detail['video_id']}")
    print(f"Question: {detail['question']}")
    print(f"Answer: {detail['answer']}")
    print(f"Score: {detail['score']}/4")
    print(f"Reason: {detail['reason']}")
    print("-" * 80 + "\n")

ASSESSMENT RESULTS
{
  "assessorProfile": {
    "id": 1,
    "name": "AI Auto-Assessor",
    "photoUrl": "https://ui-avatars.com/api/?name=AI"
  },
  "decision": "PASSED",
  "reviewedAt": "2025-11-30 10:00:00",
  "scoresOverview": {
    "project": 100,
    "interview": 12,
    "total": 80.0
  },
  "reviewChecklistResult": {
    "project": [],
    "interviews": {
      "minScore": 0,
      "maxScore": 4,
      "scores": [
        {
          "id": 1,
          "score": 3,
          "reason": "The candidate provides specific examples of challenges faced during the certification process, including issues with accuracy and violation laws. However, the answer could be more detailed in terms of the impact and outcomes of their solutions."
        },
        {
          "id": 2,
          "score": 2,
          "reason": "The candidate provided a general definition of transfer learning in TensorFlow, but did not share any specific experience or project benefits. They mentioned some pre-trained